In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.colors import rgb2hex
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix,\
    accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


In [4]:
postural_labels = ['STAND_TO_LIE', 'SIT_TO_LIE', 'LIE_TO_SIT', 'LIE_TO_STAND', 'STAND_TO_SIT', 'SIT_TO_STAND']

df = pd.read_csv('dataset.csv')
df = df.loc[df['Activity'].isin(postural_labels)]

X = df.drop(columns=['Activity'])
y = df['Activity']

dcopy = df.copy()
dcopy['Activity'] = [
    1 if a == 'STAND_TO_SIT'
    else 2 if a == 'SIT_TO_STAND'
    else 3 if a == 'SIT_TO_LIE'
    else 4 if a == 'LIE_TO_SIT'
    else 5 if a == 'STAND_TO_LIE'
    else 6
    for a in dcopy['Activity']
]

metrics = ['f1_weighted', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'accuracy']

In [5]:
# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return accuracy_score(y_true, y_pred) # return accuracy score

## KNN report

In [6]:
knn = KNeighborsClassifier(n_neighbors=9, n_jobs=-1)
acc = cross_val_score(knn, X=X, y=y, cv=10,
                      scoring=make_scorer(
                          classification_report_with_accuracy_score
                      ))

print("Accuracy")
print("mean = {mean:.2f}%;\tstd = {std:.2f};"
      .format(mean = acc.mean()*100,
              std = acc.std()))

# Average values in classification report for all folds in a K-fold Cross-validation
print(classification_report(originalclass, predictedclass))

Accuracy
mean = 75.29%;	std = 0.03;
              precision    recall  f1-score   support

  LIE_TO_SIT       0.62      0.82      0.71        85
LIE_TO_STAND       0.73      0.45      0.56        84
  SIT_TO_LIE       0.69      0.85      0.76       107
SIT_TO_STAND       0.94      0.97      0.96        33
STAND_TO_LIE       0.84      0.70      0.76       139
STAND_TO_SIT       0.86      0.89      0.87        70

    accuracy                           0.75       518
   macro avg       0.78      0.78      0.77       518
weighted avg       0.76      0.75      0.75       518



## MLP Report

In [8]:
originalclass = []
predictedclass = []

mlp = MLPClassifier(activation = 'logistic',
                    solver='adam',
                    alpha=1e-5,
                    learning_rate = 'invscaling',
                    max_iter = 1600,
                    hidden_layer_sizes=(125,),
                    random_state=42)

acc = cross_val_score(mlp, X=X, y=y, cv=10,
                      scoring=make_scorer(
                          classification_report_with_accuracy_score
                      ))

print("Accuracy")
print("mean = {mean:.2f}%;\tstd = {std:.2f};"
      .format(mean = acc.mean()*100,
              std = acc.std()))

# Average values in classification report for all folds in a K-fold Cross-validation
print(classification_report(originalclass, predictedclass))

Accuracy
mean = 74.74%;	std = 0.07;
              precision    recall  f1-score   support

  LIE_TO_SIT       0.68      0.69      0.69        85
LIE_TO_STAND       0.69      0.65      0.67        84
  SIT_TO_LIE       0.68      0.72      0.70       107
SIT_TO_STAND       0.89      0.94      0.91        33
STAND_TO_LIE       0.76      0.75      0.76       139
STAND_TO_SIT       0.91      0.87      0.89        70

    accuracy                           0.75       518
   macro avg       0.77      0.77      0.77       518
weighted avg       0.75      0.75      0.75       518

